<a href="https://colab.research.google.com/github/satojkovic/runnable_tutorials/blob/main/keras_gradcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
model_builder = keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

In [ ]:
model = model_builder(weights='imagenet')

In [ ]:
model.summary()

In [ ]:
last_conv_layer_name = 'block14_sepconv2_act'
img_path = keras.utils.get_file(
    'african_elephant.jpg', 'https://i.imgur.com/Bvro0YD.png'
)

In [ ]:
display(Image(img_path))

In [ ]:
def get_img_array(img_path, size):
  # `img` is a PIL image of size 299x299
  img = keras.preprocessing.image.load_img(img_path, target_size=size)
  # `array` is a floa32 numpy array of shape 299x299x3
  array = keras.preprocessing.image.img_to_array(img)
  # Add batch dimension
  array = np.expand_dims(array, axis=0)

  return array

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
  # outputs=[the activation of the last conv layer, the output predictions]
  grad_model = tf.keras.models.Model(
      inputs=[model.inputs], outputs=[model.get_layer(last_conv_layer_name).output, model.output]
  )
  # Compute the gradient of the top predicted class for our input image
  # with respect to the activations of the last conv layer
  with tf.GradientTape() as tape:
    last_conv_layer_output, preds = grad_model(img_array)
    if pred_index is None:
      pred_index = tf.argmax(preds[0])
    class_channel = preds[:, pred_index]

  print('last_conv_layer_output:', last_conv_layer_output.get_shape().as_list())
  print('preds:', preds.get_shape().as_list())
  tf.print('class_chanel value:', class_channel)

  # This is the gradient of the output neuron
  # with regard to the output feature map of the last conv layer
  grads = tape.gradient(class_channel, last_conv_layer_output)
  print('grads:', tf.shape(grads))

  # This is a vector where each entry is the mean intensity of the gradient
  # over a specific feature map channel
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
  print('pooled_grads:', tf.shape(pooled_grads))

  # We multiply each channel in the feature map array
  # by "how important this channel is" with regard to top predicted class
  # then sum all the channels to obtain the heatmap class activation
  last_conv_layer_output = last_conv_layer_output[0]
  heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
  print('heatmap:', tf.shape(heatmap))
  heatmap = tf.squeeze(heatmap)
  print('heatmap(squeeze)', tf.shape(heatmap))

  # For visualization purpose, we will also normalize the heatmap between [0, 1]
  heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

  return heatmap.numpy()

In [ ]:
from tensorflow.python.ops.gen_nn_ops import top_k
img_array = preprocess_input(get_img_array(img_path, size=img_size))
model = model_builder(weights='imagenet')

# Remove last layer's softmax
model.layers[-1].activation = None

preds = model.predict(img_array)
print('Predicted:', decode_predictions(preds, top=1)[0])

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

plt.matshow(heatmap)
plt.show()

In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path='cam.jpg', alpha=0.4):
  img = keras.preprocessing.image.load_img(img_path)
  img = keras.preprocessing.image.img_to_array(img)

  heatmap = np.uint8(255 * heatmap)
  jet = cm.get_cmap('jet')

  # Use RGB values of the colormap
  jet_colors = jet(np.arange(256))[:, :3]
  jet_heatmap = jet_colors[heatmap]

  # Create an image with colorized heatmap
  jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
  jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
  jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

  # Superimpose heatmap on the orignal image
  superimposed_img = jet_heatmap * alpha + img
  superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

  # Save the superimposed image
  superimposed_img.save(cam_path)

  # Display
  display(Image(cam_path))

In [ ]:
save_and_display_gradcam(img_path, heatmap)

In [ ]:
img_path = keras.utils.get_file(
    "cat_and_dog.jpg",
    "https://storage.googleapis.com/petbacker/images/blog/2017/dog-and-cat-cover.jpg",
)

display(Image(img_path))

# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Print what the two top predicted classes are
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=2)[0])

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=260)

save_and_display_gradcam(img_path, heatmap)

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=285)

save_and_display_gradcam(img_path, heatmap)